In [1]:
import pandas as pd
import numpy as np
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashVLN
import thermo
import chemicals
from thermo.interaction_parameters import IPDB
import pint
import copy
import timeit


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

global UREG
UREG = pint.UnitRegistry()

In [2]:
def cas_to_name(cas):
    if not isinstance(cas, str):
        raise ValueError("Input must be a string")    
    return ChemicalConstantsPackage.constants_from_IDs([cas]).names[0]

def name_to_cas(name):
    if not isinstance(name, str):
        raise ValueError("Input must be a string")    
    return ChemicalConstantsPackage.constants_from_IDs([name]).CASs[0]

def K_to_R(K):
    return (K - 273.15) * 1.8 + 491.67

def R_to_K(R):
    return (R - 491.67) / 1.8 + 273.15

def F_to_R(F):
    return F + 459.67

def R_to_F(R):
    return R - 459.67

def C_to_K(C):
    return C + 273.15

def F_to_C(F):
    return (F - 32) * 5 / 9

def K_to_C(K):
    return K - 273.15

def gal_to_bbl(gal):
    return gal / 42

def bbl_to_gal(bbl):
    return bbl * 42

def psi_to_Pa(psi):
    return psi * 6894.745

def bar_to_psi(bar):
    return bar * 14.5038

def Pa_to_bar(Pa):
    return Pa / 100000

def Pa_to_psi(pa):
    return pa / 6894.745

def Pa_to_mmHg(Pa):
    return Pa / 133.322

def mmHg_to_Pa(mmHg):
    return mmHg / 0.0075

def F_to_K(F):
    return (F - 32) * 5/9 + 273.15

def K_to_F(K):
    return (K - 273.15) * 1.8 + 32

In [3]:
def heating_value_to_heat_of_combustion(hv, mw):
    """
    hv = heating value, gross or net [Btu/lb]
    mw = molecular weight, g/mol
    return = heat of combustion, J/mol
    """
    
    ureg = pint.UnitRegistry()

    molecular_weight = mw * ureg('g/mol') 
    heating_value = hv * ureg('Btu/lb')   

    heat_of_combustion = heating_value * molecular_weight
    heat_of_combustion = heat_of_combustion.to('J/mol')
    return -heat_of_combustion._magnitude

# Property Update

In [4]:
df = pd.read_pickle("GPA 2145-16 Compound Properties Table - English.pkl")  

In [5]:
df.head(2)

,Order,Compound,CAS,Formula,Molar Mass [g/mol],Boiling T. [F],Triple Point T. [F],Vapor P. @60F [psia],Vapor P. @100F [psia],Crit T. [F],Crit. P. [psia],Density [lbm/ft^3],h,Liq. Relative Density @60F:1atm,API Gravity @60F:1atm,Desntiy of Liquid @60F:1atm [lbm/gal],T. Coef. of Density @60F:sat [1/F],Ideal Gas Relative Density @60F:1atm,Ideal Gas Volume @60F:1atm [ft^3/lbm],Ideal Gas Density @60F:1atm [lbm/MSCF],Summation Factor: z=1-P*b^2 @60F [1/psia^0.5],"Summation Factor: z=1-P/P0*b^2, P0=1atm @60F",Compressibility Factor @60F,Gross Heating Value Liquid [Btu/lbm],Gross Heating Value Liquid [Btu/gal],Gross Heating Value Ideal Gas [Btu/lbm],Gross Heating Value Ideal Gas [Btu/ft^3],Gross Heating Value Ideal Gas [Btu/gal],Net Heating Value Liquid [Btu/lbm],Net Heating Value Ideal Gas [Btu/ft^3],Volume of Air Required to Burn One Vol. of Ideal Gas,Heat of Vaporization @1atm [Btu/lbm],"Specific Heat Cp, ideal gas @15C [Btu/(lbm*F)]","Specific Heat Cv, ideal gas @15C [Btu/(lbm*F)]","Specific Heat Csat., liquid @15C [Btu/(lbm*F)]",k=Cp/Cv,"Refractive Index, nD @15C","Flammability Limits Lower @100F,1atm [volume % in air]","Flammability Limits Upper @100F,1atm [volume % in air]",Octane Number - Motor Method D-357,Octane Number - Research Method D-908
0,1,methane,74-82-8,CH4,16.0425,-258.66,-296.42,NaN,5000.0,-116.66,667.1,10.154,0.0114,0.30000,340.00,2.5000,NaN,0.5539,23.6549,42.27,0.0116,0.04438,NaN,NaN,NaN,23892.0,1010.0,59729.0,NaN,909.4,9.552,219.6,0.5266,0.4028,NaN,1.3073,1.00042,5.0,15.0,NaN,NaN
1,2,ethane,74-84-0,C2H6,30.0690,-127.44,-297.01,495.62,800.0,89.91,706.7,12.871,0.0995,0.35628,265.66,2.9704,-0.00733,1.0382,12.6204,79.24,0.0238,0.09130,0.5788,22185.0,65897.0,22334.0,1769.7,66340.0,20281.0,1619.0,16.715,210.4,0.4079,0.3418,0.9664,1.1932,1.00072,2.9,13.0,0.05,1.6


In [6]:
comp = dict([
('methane', 1),
('ethane', 0),
#('uranium', 0.1),
])

total_comp = sum(comp.values())
if total_comp > 1:
    comp = {k: v / total_comp for k, v in comp.items()}
zs = list(comp.values())

In [7]:
a = df[df['CAS'] == '7727-37-9']

In [8]:
pd.isna(a['Gross Heating Value Ideal Gas [Btu/ft^3]'])

16    True
Name: Gross Heating Value Ideal Gas [Btu/ft^3], dtype: bool

In [9]:
constants, properties = ChemicalConstantsPackage.from_IDs(comp.keys())

CASs = constants.CASs

# Property override with GPA standards
Pcs_updated = copy.deepcopy(constants.Pcs)
Tcs_updated = copy.deepcopy(constants.Tcs)
omegas_updated = copy.deepcopy(constants.omegas)
MWs_updated = copy.deepcopy(constants.MWs)
Hcs_updated = copy.deepcopy(constants.Hcs)

for i, cas in enumerate(CASs):
    matching_row = df[df['CAS'] == cas]
    if not matching_row.empty:
        Pcs_updated[i] = psi_to_Pa(float(matching_row['Crit. P. [psia]'].iloc[0]))
        Tcs_updated[i] = F_to_K(float(matching_row['Crit T. [F]'].iloc[0]))
        omegas_updated[i] = float(matching_row['h'].iloc[0])
        MWs_updated[i] = float(matching_row['Molar Mass [g/mol]'].iloc[0])
        Hcs_updated[i] = round(heating_value_to_heat_of_combustion(
            hv=float(matching_row['Gross Heating Value Ideal Gas [Btu/lbm]'].iloc[0]), 
            mw=MWs_updated[i]
        ), 1)

print(constants.Hcs)
constants = constants.with_new_constants(
    Pcs=Pcs_updated,
    Tcs=Tcs_updated,
    omegas=omegas_updated,
    MWs=MWs_updated,
    Hcs=Hcs_updated,
)
print(constants.Hcs)

#constants, properties = ChemicalConstantsPackage.from_IDs(comp.keys())
kijs = IPDB.get_ip_asymmetric_matrix('ChemSep PR', constants.CASs, 'kij')
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas, kijs=kijs)
gas = CEOSGas(PRMIX, eos_kwargs)
liq = CEOSLiquid(PRMIX, eos_kwargs)
flashN = FlashVLN(constants, properties, liquids=[liq, liq], gas=gas) # if dealing with water, fill [liq, liq] twice.

P = psi_to_Pa(14.7)
T = F_to_K(60)

res = flashN.flash(T=T, P=P, zs=zs)
UREG('%.15f joule/m^3' % res.Hc_standard()).to('Btu/ft^3')._magnitude * -1

[-890590.0, -1560643.0]
[-891526.6, -1562051.2]


1011.9696443283165

# Solver

In [10]:
df = pd.read_pickle("GPA 2145-16 Compound Properties Table - English.pkl")  

In [11]:
df.head(2)

,Order,Compound,CAS,Formula,Molar Mass [g/mol],Boiling T. [F],Triple Point T. [F],Vapor P. @60F [psia],Vapor P. @100F [psia],Crit T. [F],Crit. P. [psia],Density [lbm/ft^3],h,Liq. Relative Density @60F:1atm,API Gravity @60F:1atm,Desntiy of Liquid @60F:1atm [lbm/gal],T. Coef. of Density @60F:sat [1/F],Ideal Gas Relative Density @60F:1atm,Ideal Gas Volume @60F:1atm [ft^3/lbm],Ideal Gas Density @60F:1atm [lbm/MSCF],Summation Factor: z=1-P*b^2 @60F [1/psia^0.5],"Summation Factor: z=1-P/P0*b^2, P0=1atm @60F",Compressibility Factor @60F,Gross Heating Value Liquid [Btu/lbm],Gross Heating Value Liquid [Btu/gal],Gross Heating Value Ideal Gas [Btu/lbm],Gross Heating Value Ideal Gas [Btu/ft^3],Gross Heating Value Ideal Gas [Btu/gal],Net Heating Value Liquid [Btu/lbm],Net Heating Value Ideal Gas [Btu/ft^3],Volume of Air Required to Burn One Vol. of Ideal Gas,Heat of Vaporization @1atm [Btu/lbm],"Specific Heat Cp, ideal gas @15C [Btu/(lbm*F)]","Specific Heat Cv, ideal gas @15C [Btu/(lbm*F)]","Specific Heat Csat., liquid @15C [Btu/(lbm*F)]",k=Cp/Cv,"Refractive Index, nD @15C","Flammability Limits Lower @100F,1atm [volume % in air]","Flammability Limits Upper @100F,1atm [volume % in air]",Octane Number - Motor Method D-357,Octane Number - Research Method D-908
0,1,methane,74-82-8,CH4,16.0425,-258.66,-296.42,NaN,5000.0,-116.66,667.1,10.154,0.0114,0.30000,340.00,2.5000,NaN,0.5539,23.6549,42.27,0.0116,0.04438,NaN,NaN,NaN,23892.0,1010.0,59729.0,NaN,909.4,9.552,219.6,0.5266,0.4028,NaN,1.3073,1.00042,5.0,15.0,NaN,NaN
1,2,ethane,74-84-0,C2H6,30.0690,-127.44,-297.01,495.62,800.0,89.91,706.7,12.871,0.0995,0.35628,265.66,2.9704,-0.00733,1.0382,12.6204,79.24,0.0238,0.09130,0.5788,22185.0,65897.0,22334.0,1769.7,66340.0,20281.0,1619.0,16.715,210.4,0.4079,0.3418,0.9664,1.1932,1.00072,2.9,13.0,0.05,1.6


In [12]:
comp = dict([
('methane', 1),
#('methylamine', 1),
#('ethane', 1),
('docosane', 1),
])

total_comp = sum(comp.values())
if total_comp > 1:
    comp = {k: v / total_comp for k, v in comp.items()}
zs = list(comp.values())

In [13]:
constants = ChemicalConstantsPackage.constants_from_IDs(comp.keys())

CASs = constants.CASs

In [14]:
ghvs = []
for i, cas in enumerate(CASs):
    matching_row = df[df['CAS'] == cas]
    if not matching_row.empty:
        ghv = matching_row['Gross Heating Value Ideal Gas [Btu/ft^3]'].iloc[0]
        ghvs.append(ghv)
    else:
        unmatched_constants, unmatched_properties = ChemicalConstantsPackage.from_IDs([cas])
        eos_kwargs = dict(Tcs=unmatched_constants.Tcs, Pcs=unmatched_constants.Pcs, omegas=unmatched_constants.omegas)
        gas = CEOSGas(PRMIX, eos_kwargs)
        liq = CEOSLiquid(PRMIX, eos_kwargs)
        flashN = FlashVLN(unmatched_constants, unmatched_properties, liquids=[liq, liq], gas=gas)
        res = flashN.flash(T=288.706, P=101325, zs=zs) # 60F, 1atm
        
        print(unmatched_constants.names)
        print(unmatched_constants.Hcs)
        
        ghv = round(UREG('%.15f joule/m^3' % res.Hc_lower_standard()).to('Btu/ft^3')._magnitude * -1, 1)
        ghvs.append(ghv)
ghvs

['docosane']
[None]


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [ ]:
res.ws()

In [ ]:
unmatched_constants.names

In [ ]:
start_time = timeit.default_timer()

def normalize_composition(comp_dict):
    total_comp = sum(comp_dict.values())
    if total_comp > 1:
        comp_dict = {k: v / total_comp for k, v in comp_dict.items()}
    return comp_dict

comp_dict = dict([
('methane', 1),
('methylamine', 1),
('ethane', 1),
('water', 1),
    ('docosane', 1),
    ('uranium', 1)
])

comp_dict = normalize_composition(comp_dict)
comps = list(comp_dict.keys())
zs = list(comp_dict.values())

a = ChemicalConstantsPackage.constants_from_IDs(comps)

print('validity', timeit.default_timer() - start_time)

In [ ]:
a.Hcs

In [ ]:
comp_dict = dict([
    ('methane', 3),
    ('docosane', 6),
    ('uranium', 0.1),
    #('fractions', 0.7),
])

comp_dict = normalize_composition(comp_dict)
comps = list(comp_dict.keys())
zs = list(comp_dict.values())

constants = ChemicalConstantsPackage.constants_from_IDs(comps)

In [ ]:
constants

In [ ]:
constants.Hcs

In [ ]:
cas

In [ ]:
comp = dict([
('methane', 1),
#('methylamine', 1),
#('ethane', 1),
#('docosane', 1),
])

total_comp = sum(comp.values())
if total_comp > 1:
    comp = {k: v / total_comp for k, v in comp.items()}
zs = list(comp.values())


unmatched_constants, unmatched_properties = ChemicalConstantsPackage.from_IDs(comp)
eos_kwargs = dict(Tcs=unmatched_constants.Tcs, Pcs=unmatched_constants.Pcs, omegas=unmatched_constants.omegas)
gas = CEOSGas(PRMIX, eos_kwargs)
liq = CEOSLiquid(PRMIX, eos_kwargs)
flashN = FlashVLN(unmatched_constants, unmatched_properties, liquids=[liq, liq], gas=gas)
res = flashN.flash(T=288.706, P=101325, zs=zs) # 60F, 1atm

In [ ]:
res.Hcs

In [ ]:
res.Hc_standard()

In [ ]:
round(UREG('%.15f joule/m^3' % res.Hc_standard()).to('Btu/ft^3')._magnitude * -1, 6)

In [ ]:
from fluids.constants import R

In [ ]:
R

# Figure out what to do with compounds that are not identified. 

Probably do correlation with API and specific gravity with HHV and LHV. 

# MW / 28.95 for specific gravity is valid for standard conditions. I need to find changing specific gravity models for non-standard conditions. But sg_standard would still work for characterization of petroleum fractions

In [ ]:
comp = dict([
#('methane', 1),
#('methylamine', 1),
#('ethane', 1),
#('methylcyclohexane', 1),
('docosane', 1),
])



total_comp = sum(comp.values())
if total_comp > 1:
    comp = {k: v / total_comp for k, v in comp.items()}
zs = list(comp.values())

constants = ChemicalConstantsPackage.constants_from_IDs(comp.keys())
print('Tbs =', constants.Tbs[0])
print('Tcs =', constants.Tcs[0])
print('Pcs =', constants.Pcs[0])
print('Hcs =', constants.Hcs[0])
print('omegas =', constants.omegas[0])

In [ ]:
constants

In [ ]:
def heating_value_to_heat_of_combustion(hv, mw):
    """
    hv = heating value, gross or net [Btu/lb]
    mw = molecular weight, g/mol
    return = heat of combustion, J/mol
    """
    
    ureg = pint.UnitRegistry()

    molecular_weight = mw * ureg('g/mol') 
    heating_value = hv * ureg('Btu/lb')   

    heat_of_combustion = heating_value * molecular_weight
    heat_of_combustion = heat_of_combustion.to('J/mol')
    return -heat_of_combustion._magnitude

In [ ]:
henei = '629-94-7'
df[df['CAS'] == '629-78-7']

In [ ]:
# https://en.wikipedia.org/wiki/Higher_alkane
no_GPA_match = ['heneicosane', 'heptadecane', '']
no_Hcs = ['heneicosane']

In [ ]:
constants, properties = ChemicalConstantsPackage.from_IDs(['heneicosane'])
constants

In [ ]:
pd.DataFrame(data=[[1, 2,3,4, 5]])

In [ ]:
import pandas as pd
import numpy as np
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashVLN
import thermo
import chemicals
from thermo.interaction_parameters import IPDB
import pint
import copy
import fluids
import config
import timeit
from scipy.optimize import newton
import correlations


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ureg = pint.UnitRegistry()

settings = {
    "T_STANDARD": 288.70555,  # Temperature in Kelvin, 60F
    "P_STANDARD": 101325.0,    # Pressure in Pascal, 1 atm
    "R": 8.31446261815324,
}
config.update_config(settings)


def normalize_composition(comp_dict):
    """
    :param comp_dict: un-normalized dictionary of composition. {"CH4": 3, "C2H6", 6}
    :return: normalized dictionary of composition. {"CH4": 0.3333, "C2H6", 0.66666666}
    """
    total_comp = sum(comp_dict.values())
    if total_comp > 0:
        keys = list(comp_dict.keys())
        last_key = keys[-1]
        normalized_values = [v / total_comp for v in comp_dict.values()]

        # Normalize all but the last element
        comp_dict = {keys[i]: normalized_values[i] for i in range(len(keys) - 1)}

        # Adjust the last element so that the sum is exactly 1
        comp_dict[last_key] = 1 - sum(comp_dict.values())

    return comp_dict


def check_properties_exists(constants):
    """
    :param constants: constants object of the thermo library
    Molecular weight and normal boiling points are minimum information needed to characterize a fluid, incase of any
    missing properties. For example, docosane is missing Heat of combustion (J/mol), but it can be correlated.
    """
    rhol_60Fs_mass = constants.rhol_60Fs_mass
    mws = constants.MWs
    Tbs = constants.Tbs
    names = constants.names
    Hcs = constants.Hcs

    # Todo: implement a method to add missing data.
    for rhol_60F_mass, mw, Tb, Hc, name in zip(rhol_60Fs_mass, mws, Tbs, Hcs, names):
        if rhol_60F_mass is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (rhol_60F_mass, liquid mass density at 60F)." % name)
        if mw is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (mw, molecular weight)." % name)
        if Tb is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (Tb, normal boiling temperature)." % name)
        if Hc is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (Hc, heat of combustion [J/mol])." % name)


def ideal_gas_molar_volume():
    """
    PV=nRT, where number of moles n=1. Rearranging -> V=RT/P
    R = 8.31446261815324 ((m^3-Pa)/(mol-K))
    T = 288.7056 K, 60F, standard temperature
    P = 101325 Pa, 1 atm, standard pressure
    :return: ideal gas molar volume in a standard condition (m^3/mol)
    """
    return config.constants['R'] * config.constants['T_STANDARD'] / config.constants['P_STANDARD']


def is_fraction(s):
    """
    string detector for petroleum fractions. Specialized codes for fractions are triggered if detected.
    """
    return s.lower() in ['fraction', 'fractions']

def get_ghvs_pure_compounds(constants, df_GPA):
    """
    :param constants: thermo's constants object
    :param GPA_data: pandas dataframe of the GPA 2145-16 Table
    :return:
    """
    ghvs_ideal_gas = []
    df = df_GPA
    V_molar = ideal_gas_molar_volume()  # fixed 0.0236 m^3/mol at standard conditions for all compounds
    for cas, name, Hc, rhol_60F_mass in zip(constants.CASs, constants.names, constants.Hcs, constants.rhol_60Fs_mass):

        matching_row = df[df['CAS'] == cas]

        # chemical is found in the GPA data table
        if not matching_row.empty:
            ghv_ideal_gas = matching_row['Gross Heating Value Ideal Gas [Btu/ft^3]'].iloc[0]

            if pd.isna(ghv_ideal_gas):

                # chemically inert or contains to combustible energy. Skip them. Ex: nitrogen, argon, helium
                if Hc != 0:
                    ghv_ideal_gas = Hc / V_molar
                    ghv_ideal_gas = ureg('%.15f joule/m^3' % ghv_ideal_gas).to('Btu/ft^3')._magnitude * -1
                else:
                    ghv_ideal_gas = 0

        # chemical is NOT identified in the GPA datatable
        else:

            if Hc == 0:  # chemically inert or contains to combustible energy.
                ghv_ideal_gas = 0
            elif Hc is None:
                """
                sg_liq = correlations.sg_liq(rhol_60F_mass)
                API = newton(lambda API: correlations.API_sg_liq(API, sg_liq), x0=45, maxiter=500)

                # ghv_liquid correlation usig API.

                """
                pass  # Todo: implement a handler that can correlate API to ghv for a working range of the model. Show a warning sign if outside range. Prompt the user to activate ghv_correlate=True
            else:
                ghv_ideal_gas = Hc / V_molar
                ghv_ideal_gas = ureg('%.15f joule/m^3' % ghv_ideal_gas).to('Btu/ft^3')._magnitude * -1

        ghvs_ideal_gas.append(ghv_ideal_gas)
    return np.array(ghvs_ideal_gas)


df = pd.read_pickle("GPA 2145-16 Compound Properties Table - English.pkl")

#df = df[df['Compound'] != 'nitrogen']

comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 2.304),
    ('carbon dioxide', 1.505),
    ('methane', 71.432),
    ('ethane', 11.732),
    ('propane', 7.595),
    ('isobutane', 0.827),
    ('n-butane', 2.540),
    ('i-pentane', 0.578),
    ('n-pentane', 0.597),
    ('fractions', 0.889),
])
comp_dict = normalize_composition(comp_dict)

comp_dict_pure = {}
comp_dict_fraction = {}
for key, value in comp_dict.items():
    if is_fraction(key):
        comp_dict_fraction[key] = value
    else:
        comp_dict_pure[key] = value

comps_pure = list(comp_dict_pure.keys())
zs_pure = np.array(list(comp_dict_pure.values()))

constants_pure = ChemicalConstantsPackage.constants_from_IDs(comps_pure)

# check if the compounds have molecular weight and normal boiling T data
check_properties_exists(constants_pure)

ghvs_pure = get_ghvs_pure_compounds(constants_pure, df_GPA=df)

temp = {key: val for key, val in zip(comps_pure, ghvs_pure)}
for k, v in temp.items():
    print('%s    : %.1f' % (k, v))
print('--------------------------------------------')


df_ghvs_table = pd.DataFrame(data=[])

columns=['Compound Name', 'CAS', 'Ideal Gas GHV [Btu/scf]', 'Mole Frac.', 'Weighted GVH [Btu/scf']

print(df_ghvs_table)


In [15]:
ghvs_pure

NameError: name 'ghvs_pure' is not defined

In [16]:
zs_pure

NameError: name 'zs_pure' is not defined

In [17]:
wghtd_ghvs_pure = ghvs_pure * zs_pure

NameError: name 'ghvs_pure' is not defined

In [81]:

# Brazos Gas
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 2.304),
    ('carbon dioxide', 1.505),
    ('methane', 71.432),
    ('ethane', 11.732),
    ('propane', 7.595),
    ('isobutane', 0.827),
    ('n-butane', 2.540),
    ('i-pentane', 0.578),
    ('n-pentane', 0.597),
    ('fractions', 0.889),
])

# StateCordell VRU
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0.1590),
    ('carbon dioxide', 0.6910),
    ('methane', 51.0870),
    ('ethane', 19.9110),
    ('propane', 14.8830),
    ('isobutane', 2.6940),
    ('n-butane', 6.0390),
    ('isopentane', 1.4840),
    ('n-pentane', 1.5790),
    ('fractions', 1.4720),
])

# Thurmond
comp_dict = dict([
    ('carbon dioxide', 0.261),
    ('nitrogen', 1.295),
    ('methane', 86.878),
    ('ethane', 5.659),
    ('propane', 3.237),
    ('isobutane', 0.405),
    ('n-butane', 1.051),
    ('neopentane', 0),
    ('isopentane', 0.306),
    ('n-pentane', 0.381),
    ('fractions', 0.527),
])

# Storey Ranch
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0),
    ('carbon dioxide', 0.428),
    ('methane', 1.078),
    ('ethane', 8.520),
    ('propane', 25.730),
    ('isobutane', 6.757),
    ('n-butane', 23.3999),
    ('neopentane', 0.056),
    ('i-pentane', 8.021),
    ('n-pentane', 9.569),
    ('hexane', 9.467),
    ('fractions', 6.975),
])

# Fesco - Combs, HC Liq - HT
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0.103),
    ('carbon dioxide', 1.485),
    ('methane', 34.352),
    ('ethane', 24.949),
    ('propane', 24.893),
    ('isobutane', 3.633),
    ('n-butane', 7.661),
    ('neopentane', 0.017),
    ('isopentane', 1.240),
    ('n-pentane', 1.108),
    ('hexane', 0.409),
    ('fractions', 0.15),
])

# Fesco - Combs, Sep Gas
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0.135),
    ('carbon dioxide', 1.707),
    ('methane', 47.167),
    ('ethane', 20.530),
    ('propane', 18.116),
    ('isobutane', 2.928),
    ('n-butane', 6.460),
    ('neopentane', 0.020),
    ('isopentane', 1.088),
    ('n-pentane', 0.965),
    ('hexane', 0.540),
    ('fractions', 0.344),
])

# Kendall - Mirror VRU Discharge
comp_dict = dict([
    ('hydrogen sulfide', 0.01),
    ('nitrogen', 0.0810),
    ('carbon dioxide', 0.3810),
    ('methane', 18.5830),
    ('ethane', 22.6350),
    ('propane', 30.2270),
    ('isobutane', 5.5680),
    ('n-butane', 14.5620),
    ('isopentane', 3.2510),
    ('n-pentane', 2.7770),
    ('fractions', 1.9250),
])

In [125]:
import pandas as pd
import numpy as np
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashVLN
import thermo
import chemicals
from thermo.interaction_parameters import IPDB
import pint
import copy
import fluids
import config
import timeit
from scipy.optimize import newton
import correlations


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ureg = pint.UnitRegistry()

settings = {
    "T_STANDARD": 288.70555,  # Temperature in Kelvin, 60F
    "P_STANDARD": 101325.0,    # Pressure in Pascal, 1 atm
    "R": 8.31446261815324,
}
config.update_config(settings)


def normalize_composition(comp_dict):
    """
    :param comp_dict: un-normalized dictionary of composition. {"CH4": 3, "C2H6", 6}
    :return: normalized dictionary of composition. {"CH4": 0.3333, "C2H6", 0.66666666}
    """
    total_comp = sum(comp_dict.values())
    if total_comp > 0:
        keys = list(comp_dict.keys())
        last_key = keys[-1]
        normalized_values = [v / total_comp for v in comp_dict.values()]

        # Normalize all but the last element
        comp_dict = {keys[i]: normalized_values[i] for i in range(len(keys) - 1)}

        # Adjust the last element so that the sum is exactly 1
        comp_dict[last_key] = 1 - sum(comp_dict.values())

    return comp_dict


def check_properties_exists(constants):
    """
    :param constants: constants object of the thermo library
    Molecular weight and normal boiling points are minimum information needed to characterize a fluid, incase of any
    missing properties. For example, docosane is missing Heat of combustion (J/mol), but it can be correlated.
    """
    rhol_60Fs_mass = constants.rhol_60Fs_mass
    mws = constants.MWs
    Tbs = constants.Tbs
    names = constants.names
    Hcs = constants.Hcs

    # Todo: implement a method to add missing data.
    for rhol_60F_mass, mw, Tb, Hc, name in zip(rhol_60Fs_mass, mws, Tbs, Hcs, names):
        if rhol_60F_mass is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (rhol_60F_mass, liquid mass density at 60F)." % name)
        if mw is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (mw, molecular weight)." % name)
        if Tb is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (Tb, normal boiling temperature)." % name)
        if Hc is None:
            raise ValueError("Chemical name '%s' is recognized but missing a required data (Hc, heat of combustion [J/mol])." % name)


def ideal_gas_molar_volume():
    """
    PV=nRT, where number of moles n=1. Rearranging -> V=RT/P
    R = 8.31446261815324 ((m^3-Pa)/(mol-K))
    T = 288.7056 K, 60F, standard temperature
    P = 101325 Pa, 1 atm, standard pressure
    :return: ideal gas molar volume in a standard condition (m^3/mol)
    """
    return config.constants['R'] * config.constants['T_STANDARD'] / config.constants['P_STANDARD']


def is_fraction(s):
    """
    string detector for petroleum fractions. Specialized codes for fractions are triggered if detected.
    """
    return s.lower() in ['fraction', 'fractions']

def get_ghvs_pure_compounds(constants, df_GPA):
    """
    :param constants: thermo's constants object
    :param GPA_data: pandas dataframe of the GPA 2145-16 Table
    :return:
    """
    ghvs_ideal_gas = []
    df = df_GPA
    V_molar = ideal_gas_molar_volume()  # fixed 0.0236 m^3/mol at standard conditions for all compounds
    for cas, name, Hc, rhol_60F_mass in zip(constants.CASs, constants.names, constants.Hcs, constants.rhol_60Fs_mass):

        matching_row = df[df['CAS'] == cas]

        # chemical is found in the GPA data table
        if not matching_row.empty:
            ghv_ideal_gas = matching_row['Gross Heating Value Ideal Gas [Btu/ft^3]'].iloc[0]

            if pd.isna(ghv_ideal_gas):

                # chemically inert or contains to combustible energy. Skip them. Ex: nitrogen, argon, helium
                if Hc != 0:
                    ghv_ideal_gas = Hc / V_molar
                    ghv_ideal_gas = ureg('%.15f joule/m^3' % ghv_ideal_gas).to('Btu/ft^3')._magnitude * -1
                else:
                    ghv_ideal_gas = 0

        # chemical is NOT identified in the GPA datatable
        else:

            if Hc == 0:  # chemically inert or contains to combustible energy.
                ghv_ideal_gas = 0
            elif Hc is None:
                """
                sg_liq = correlations.sg_liq(rhol_60F_mass)
                API = newton(lambda API: correlations.API_sg_liq(API, sg_liq), x0=45, maxiter=500)

                # ghv_liquid correlation usig API.

                """
                pass  # Todo: implement a handler that can correlate API to ghv for a working range of the model. Show a warning sign if outside range. Prompt the user to activate ghv_correlate=True
            else:
                ghv_ideal_gas = Hc / V_molar
                ghv_ideal_gas = ureg('%.15f joule/m^3' % ghv_ideal_gas).to('Btu/ft^3')._magnitude * -1

        ghvs_ideal_gas.append(ghv_ideal_gas)
    return np.array(ghvs_ideal_gas)


df = pd.read_pickle("GPA 2145-16 Compound Properties Table - English.pkl")

#df = df[df['Compound'] != 'nitrogen']

# StateCordell VRU
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0.1590),
    ('carbon dioxide', 0.6910),
    ('methane', 51.0870),
    ('ethane', 19.9110),
    ('propane', 14.8830),
    ('isobutane', 2.6940),
    ('n-butane', 6.0390),
    ('isopentane', 1.4840),
    ('n-pentane', 1.5790),
    ('fractions', 1.4720),
])

# Thurmond
comp_dict = dict([
    ('carbon dioxide', 0.261),
    ('nitrogen', 1.295),
    ('methane', 86.878),
    ('ethane', 5.659),
    ('propane', 3.237),
    ('isobutane', 0.405),
    ('n-butane', 1.051),
    ('neopentane', 0),
    ('isopentane', 0.306),
    ('n-pentane', 0.381),
    ('fractions', 0.527),
])

# Fesco - Combs, HC Liq - HT
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0.103),
    ('carbon dioxide', 1.485),
    ('methane', 34.352),
    ('ethane', 24.949),
    ('propane', 24.893),
    ('isobutane', 3.633),
    ('n-butane', 7.661),
    ('neopentane', 0.017),
    ('isopentane', 1.240),
    ('n-pentane', 1.108),
    ('hexane', 0.409),
    ('fractions', 0.15),
])

# Fesco - Combs, Sep Gas
comp_dict = dict([
    ('hydrogen sulfide', 0),
    ('nitrogen', 0.135),
    ('carbon dioxide', 1.707),
    ('methane', 47.167),
    ('ethane', 20.530),
    ('propane', 18.116),
    ('isobutane', 2.928),
    ('n-butane', 6.460),
    ('neopentane', 0.020),
    ('isopentane', 1.088),
    ('n-pentane', 0.965),
    ('hexane', 0.540),
    ('fractions', 0.344),
])

# Storey Ranch
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 0),
    ('carbon dioxide', 0.428),
    ('methane', 1.078),
    ('ethane', 8.520),
    ('propane', 25.730),
    ('isobutane', 6.757),
    ('n-butane', 23.3999),
    ('neopentane', 0.056),
    ('i-pentane', 8.021),
    ('n-pentane', 9.569),
    ('hexane', 9.467),
    ('fractions', 6.975),
])

# Kendall - Mirror VRU Discharge
comp_dict = dict([
    ('hydrogen sulfide', 0.01),
    ('nitrogen', 0.0810),
    ('carbon dioxide', 0.3810),
    ('methane', 18.5830),
    ('ethane', 22.6350),
    ('propane', 30.2270),
    ('isobutane', 5.5680),
    ('n-butane', 14.5620),
    ('isopentane', 3.2510),
    ('n-pentane', 2.7770),
    ('fractions', 1.9250),
])

# Brazos Gas
comp_dict = dict([
    ('hydrogen sulfide', 0.001),
    ('nitrogen', 2.304),
    ('carbon dioxide', 1.505),
    ('methane', 71.432),
    ('ethane', 11.732),
    ('propane', 7.595),
    ('isobutane', 0.827),
    ('n-butane', 2.540),
    ('i-pentane', 0.578),
    ('n-pentane', 0.597),
    ('fractions', 0.889),
])

comp_dict = normalize_composition(comp_dict)

comp_dict_pure = {}
comp_dict_fraction = {}
for key, value in comp_dict.items():
    if is_fraction(key):
        comp_dict_fraction[key] = value
    else:
        comp_dict_pure[key] = value

comps_pure = list(comp_dict_pure.keys())
zs_pure = np.array(list(comp_dict_pure.values()))
zs_fraction = list(comp_dict_fraction.values())[0]

constants_pure = ChemicalConstantsPackage.constants_from_IDs(comps_pure)

# check if the compounds have molecular weight and normal boiling T data
check_properties_exists(constants_pure)

ghvs_pure = get_ghvs_pure_compounds(constants_pure, df_GPA=df)
wghtd_ghvs_pure = ghvs_pure * zs_pure

ghv = 1320.1

ghv_fraction = (ghv - sum(wghtd_ghvs_pure)) / zs_fraction
whgtd_ghv_fraction = ghv_fraction * zs_fraction

###########

fraction_row = ['fraction', None, ghv_fraction, zs_fraction, whgtd_ghv_fraction]
sum_row = ['Total', None, None, (sum(zs_pure) + zs_fraction) * 100, sum(wghtd_ghvs_pure) + whgtd_ghv_fraction]

df_ghvs_table = pd.DataFrame(data=[comps_pure, constants_pure.CASs, ghvs_pure, zs_pure * 100, wghtd_ghvs_pure]).T
df_ghvs_table.columns = ['Compound Name', 'CAS', 'Ideal Gas GHV [Btu/scf]', 'Mole Frac. [%]', 'Wghtd. Ideal Gas GVH [Btu/scf]']
df_ghvs_table.loc[len(df_ghvs_table)] = fraction_row
df_ghvs_table.loc[len(df_ghvs_table)] = sum_row
df_ghvs_table


,Compound Name,CAS,Ideal Gas GHV [Btu/scf],Mole Frac. [%],Wghtd. Ideal Gas GVH [Btu/scf]
0,hydrogen sulfide,7783-06-4,637.1,0.001,0.006371
1,nitrogen,7727-37-9,0.0,2.304,0.0
2,carbon dioxide,124-38-9,0.0,1.505,0.0
3,methane,74-82-8,1010.0,71.432,721.4632
4,ethane,74-84-0,1769.7,11.732,207.621204
5,propane,74-98-6,2516.1,7.595,191.097795
6,isobutane,75-28-5,3251.9,0.827,26.893213
7,n-butane,106-97-8,3262.3,2.54,82.86242
8,i-pentane,78-78-4,4000.9,0.578,23.125202
9,n-pentane,109-66-0,4008.7,0.597,23.931939


# Build solver that allows you to pick among Sg, MW, and HHV. 

In [102]:
sum(wghtd_ghvs_pure) + whgtd_ghv_fraction

1735.999988999999

In [100]:
ghv_fraction

7507.1

In [117]:
API = 59
HHV_liq = 17721 + 89.08*API - 0.348*API**2 + 0.009518*API**3

In [118]:
mw = 96.82

In [119]:
HHV_liq / (379.5/mw)

6051.601899752412

In [21]:
V_molar = ideal_gas_molar_volume()  # m^3/mol
V_molar = ureg('%.15f m^3/mol' % V_molar).to('ft^3/mol')._magnitude  # ft^3/mol

In [22]:
[V_molar for _ in range(len(constants_pure.names))]

[0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864,
 0.8366191810583864]

In [23]:
constants

ChemicalConstantsPackage(atom_fractions=[{'C': 0.2, 'H': 0.8}, {'C': 0.3235294117647059, 'H': 0.6764705882352942}], atomss=[{'C': 1, 'H': 4}, {'C': 22, 'H': 46}], Carcinogens=[{'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}, {'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}], CASs=['74-82-8', '629-97-0'], charges=[0, 0], dipoles=[0.0, 0.0], formulas=['CH4', 'C22H46'], Gfgs=[-50443.48000000001, 133050.7749999999], Gfgs_mass=[-3144373.1198332435, 428366.06840217684], GWPs=[84.0, None], Hcs=[-890590.0, None], Hcs_lower=[-802567.008, None], Hcs_lower_mass=[-50027677.0520232, None], Hcs_mass=[-55514553.25430141, None], Hfgs=[-74534.0, -498580.0], Hfgs_mass=[-4646045.556604162, -1605212.4039409577], Hfus_Tms=[940.0, 48800.0], Hfus_Tms_mass=[58594.50483279996, 157114.93704584768], Hsub_Tts=[9669.321795841746, 154624.8922731871], Hsub

In [24]:
df_matched = df[df['CAS'].isin(CASs)]

CASs_matched = df_matched['CAS'].values
CASs_unmatched = np.setdiff1d(CASs, CASs_matched)
CASs_unmatched

array(['629-97-0'], dtype='<U8')

In [25]:

comp = dict([
    ('methane', 3),
    ('ethane', 3),
    ('hexane', 3),
    ('decane', 3),
    ('heptane', 3),
    ('docosane', 6),
    ('uranium', 0.1),
])
comp = dict([
    ('methane', 3),
    ('nitrogen', 3),
    ('docosane', 3),
    ('dodecane', 3),
])


total_comp = sum(comp.values())
if total_comp > 1:
    comp = {k: v / total_comp for k, v in comp.items()}
zs = list(comp.values())

constants, properties = ChemicalConstantsPackage.from_IDs(comp.keys())


kijs = IPDB.get_ip_asymmetric_matrix('ChemSep PR', constants.CASs, 'kij')
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas, kijs=kijs)
gas = CEOSGas(PRMIX, eos_kwargs)
liq = CEOSLiquid(PRMIX, eos_kwargs)
flashN = FlashVLN(constants, properties, liquids=[liq, liq], gas=gas) # if dealing with water, fill [liq, liq] twice.

P = psi_to_Pa(14.7)
T = F_to_K(60)

res = flashN.flash(T=T, P=P, zs=zs)

In [26]:
res.SG()

0.7466761039394252

In [27]:
res.SG_gas()

4.532257602232153

In [28]:
res.ws()

[0.030557570721071316,
 0.05335973732442901,
 0.5916300257448056,
 0.324452666209694]

In [29]:
constants

ChemicalConstantsPackage(atom_fractions=[{'C': 0.2, 'H': 0.8}, {'N': 1.0}, {'C': 0.3235294117647059, 'H': 0.6764705882352942}, {'C': 0.3157894736842105, 'H': 0.6842105263157895}], atomss=[{'C': 1, 'H': 4}, {'N': 2}, {'C': 22, 'H': 46}, {'C': 12, 'H': 26}], Carcinogens=[{'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}, {'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}, {'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}, {'International Agency for Research on Cancer': 'Unlisted', 'National Toxicology Program 13th Report on Carcinogens': 'Unlisted'}], CASs=['74-82-8', '7727-37-9', '629-97-0', '112-40-3'], charges=[0, 0, 0, 0], dipoles=[0.0, 0.0, 0.0, 0.0], formulas=['CH4', 'N2', 'C22H46', 'C12H26'], Gfgs=[-50443.48000000001, 0.0, 133050.7749999999, 5198

In [30]:
# Ideal gas heating value
ureg('%.15f joule/m^3' % res.Hc_standard()).to('Btu/ft^3')._magnitude * -1

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [ ]:
constants.Hcs[0] / constants.MWs[0] * 1000 / 1000000

In [31]:
constants.Hcs[0]

-890590.0

In [32]:
constants.MWs[0]

16.04246

In [33]:
import pint

# Setting up the unit registry
ureg = pint.UnitRegistry()

# Define the molecular weight and heating value
molecular_weight = 16.04 * ureg('g/mol')  # Molecular weight of methane in g/mol
gross_heating_value = 55.575 * ureg('MJ/kg')  # Gross heating value in MJ/kg

# Convert the heating value to kJ/g (since 1 MJ = 1000 kJ)
heating_value_kJ_per_g = gross_heating_value.to('kJ/g')

# Calculate the heat of combustion in kJ/mol
heat_of_combustion = heating_value_kJ_per_g * molecular_weight

heat_of_combustion = heat_of_combustion.to('J/mol')

heat_of_combustion

891423.0 <Unit('joule / mole')>

In [34]:
def heating_value_to_heat_of_combustion(hv, mw):
    """
    hv = heating value, gross or net [MJ/kg]
    mw = molecular weight, g/mol
    return = heat of combustion, J/mol
    """
    
    ureg = pint.UnitRegistry()

    molecular_weight = 16.04 * ureg('g/mol') 
    heating_value = 55.575 * ureg('MJ/kg')   

    heat_of_combustion = heating_value * molecular_weight
    heat_of_combustion = heat_of_combustion.to('J/mol')
    return heat_of_combustion._magnitude

In [35]:
heating_value_to_heat_of_combustion(55.575, 16.04)

891423.0